# Import libraries

In [1]:
# !pip install xgboost
# !pip install sklearn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import gc

# Load data

In [3]:
train=pd.read_table('train_data.csv', delimiter=',')
test=pd.read_table('test_data.csv', delimiter=',')

In [4]:
train=train.drop('index', axis=1) #drop index column
test=test.drop('index', axis=1)

In [5]:
train.head()

,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,31.52,...,-27.68,-37.21,8.32,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58
1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,31.52,...,-21.13,-36.57,8.77,21.17,4.44,48.60,27.41,-23.77,15.44,3.42
2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,31.52,...,-10.72,-34.16,6.99,32.16,5.01,48.53,19.21,-33.16,15.11,4.82
3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,31.52,...,0.33,-31.04,6.17,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74
4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,31.52,...,9.83,-31.80,7.47,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95


In [6]:
print('Train:', train.shape, "| Test:", test.shape)

Train: (375734, 245) | Test: (31354, 244)


In [7]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375734 entries, 0 to 375733
Columns: 245 entries, lat to wind-vwnd-925-2010-20
dtypes: float64(240), int64(3), object(2)
memory usage: 702.3+ MB
None


# EDA

In [8]:
#print columns with categorical data
train.select_dtypes(include='object').columns

Index(['startdate', 'climateregions__climateregion'], dtype='object')

# Data preprocessing

## Fill missing values

In [9]:
#Find the Null-containing columns  
null_cols = train.columns[train.isna().any()].tolist()
null_cols

['nmme0-tmp2m-34w__ccsm30',
 'nmme-tmp2m-56w__ccsm3',
 'nmme-prate-34w__ccsm3',
 'nmme0-prate-56w__ccsm30',
 'nmme0-prate-34w__ccsm30',
 'nmme-prate-56w__ccsm3',
 'nmme-tmp2m-34w__ccsm3',
 'ccsm30']

In [10]:
null_cols_test = test.columns[test.isna().any()].tolist()

In [11]:
#impute (fill) missing values using sklearn's simple imputer
for col in null_cols: 
    train[col] = train[col].interpolate()
train.columns[train.isna().any()].tolist()

[]

# Feature engineering

## Encoding regions

In [12]:
#rewrite as function later
def encode_regions(train, test): 
    encoder = LabelEncoder()
    new_climate_regions = encoder.fit_transform(train['climateregions__climateregion'])
    train['climateregions__climateregion'] = new_climate_regions
    test['climateregions__climateregion'] = encoder.transform(test['climateregions__climateregion'])
    cli_reg_map = {index: new_climate_regions for index, new_climate_regions in 
                      enumerate(encoder.classes_)}
    print(cli_reg_map)
    #Check for difference climate regions between train and test sets
    train_regions = train.climateregions__climateregion.value_counts
    test_regions = train.climateregions__climateregion.value_counts
    #print(train_regions == test_regions) #train and test on same climate regions
    #train and test set cover resembling climate regions
    return train, test

In [13]:
encode_regions(train, test)
train['climateregions__climateregion'].unique()

{0: 'BSh', 1: 'BSk', 2: 'BWh', 3: 'BWk', 4: 'Cfa', 5: 'Cfb', 6: 'Csa', 7: 'Csb', 8: 'Dfa', 9: 'Dfb', 10: 'Dfc', 11: 'Dsb', 12: 'Dsc', 13: 'Dwa', 14: 'Dwb'}


array([ 0,  4,  1,  3,  2,  6,  7,  5,  9, 12, 10,  8, 11, 13, 14])

## Location encoding

Encode location using latitude and longitude, round values to 14th decimal digits.
Reference: `https://www.kaggle.com/code/flaviafelicioni/wids-2023-different-locations-train-test-solved`

In [14]:
def group_loc(train, test):
    #Scale lat and lon to 14th decimal digit
    scale = 14
    train.loc[:,'lat'] = round(train.lat, scale)
    train.loc[:,'lon'] = round(train.lon, scale)
    test.loc[:,'lat'] = round(test.lat, scale)
    test.loc[:,'lon'] = round(test.lon, scale)
    # Concatenate train and test data
    all_df = pd.concat([train, test], axis=0)
    # Create new feature
    all_df['loc_group'] = all_df.groupby(['lat','lon']).ngroup()
    print(f'{all_df.loc_group.nunique()} unique locations')
    
    # Split back up
    train = all_df.iloc[:len(train)]
    test = all_df.iloc[len(train):]

    #Check result
    print('Locations in train that are not in test')
    locations_train=list(train.loc_group.unique())
    locations_test=list(test.loc_group.unique())
    result_1 = list(set(locations_train).difference(locations_test))
    print(result_1)

    print('Locations in test that are not in train')
    result_2=list(set(locations_test).difference(locations_train))
    print(result_2)
    
    return train, test

In [15]:
train, test = group_loc(train, test)

514 unique locations
Locations in train that are not in test
[]
Locations in test that are not in train
[]


## Cyclical date conversion

In [16]:
#add season
def add_season(df):
  month_to_season = {
      1: 0,
      2: 0,
      3: 1,
      4: 1,
      5: 1,
      6: 2,
      7: 2,
      8: 2, 
      9: 3, 
      10: 3,
      11: 3,
      12: 0
  }
  df['season'] = df['month'].apply(lambda x: month_to_season[x])

In [17]:
def convert_time(df):
    df['year'] = df['DATE'].dt.year
    df['month'] = df['DATE'].dt.month
    df['day'] = df['DATE'].dt.day_of_year
#     df['week'] = df['DATE'].dt.isocalendar().week
#     df['quarter'] = df['DATE'].dt.quarter
    return df

In [18]:
from sklearn.preprocessing import FunctionTransformer

def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

def encode_cyclical(df):
    # encode the day with a period of 365
    df['day_of_year_sin'] = sin_transformer(365).fit_transform(df['day'])
    df['day_of_year_cos'] = cos_transformer(365).fit_transform(df['day'])

#     # encode the week with a period of 52
#     df['week_sin'] = sin_transformer(52).fit_transform(df['week'])
#     df['week_cos'] = cos_transformer(52).fit_transform(df['week'])

    # encode the month with a period of 12
    df['month_sin'] = sin_transformer(12).fit_transform(df['month'])
    df['month_cos'] = cos_transformer(12).fit_transform(df['month'])

    # encode the season with a period of 4
    df['season_sin'] = sin_transformer(4).fit_transform(df['season'])
    df['season_cos'] = cos_transformer(4).fit_transform(df['season'])
    
#     # encode the quarter with a period of 4
#     df['quarter_sin'] = sin_transformer(4).fit_transform(df['quarter'])
#     df['quarter_cos'] = cos_transformer(4).fit_transform(df['quarter'])

In [19]:
#Encode start date
train['DATE'] = pd.to_datetime(train['startdate'])
test['DATE'] = pd.to_datetime(test['startdate'])

convert_time(train)
add_season(train)

convert_time(test)
add_season(test)

encode_cyclical(train)
encode_cyclical(test)

In [20]:
train.head(10)

,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,nmme0-tmp2m-34w__gfdlflora0,...,year,month,day,season,day_of_year_sin,day_of_year_cos,month_sin,month_cos,season_sin,season_cos
0,0.0,0.833333,9/1/14,237.00,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,244,3,-0.871706,-0.490029,-1.0,-1.836970e-16,-1.0,-1.836970e-16
1,0.0,0.833333,9/2/14,228.90,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,245,3,-0.880012,-0.474951,-1.0,-1.836970e-16,-1.0,-1.836970e-16
2,0.0,0.833333,9/3/14,220.69,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,246,3,-0.888057,-0.459733,-1.0,-1.836970e-16,-1.0,-1.836970e-16
3,0.0,0.833333,9/4/14,225.28,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,247,3,-0.895839,-0.444378,-1.0,-1.836970e-16,-1.0,-1.836970e-16
4,0.0,0.833333,9/5/14,237.24,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,248,3,-0.903356,-0.428892,-1.0,-1.836970e-16,-1.0,-1.836970e-16
5,0.0,0.833333,9/6/14,237.87,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,249,3,-0.910605,-0.413279,-1.0,-1.836970e-16,-1.0,-1.836970e-16
6,0.0,0.833333,9/7/14,236.36,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,250,3,-0.917584,-0.397543,-1.0,-1.836970e-16,-1.0,-1.836970e-16
7,0.0,0.833333,9/8/14,233.36,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,251,3,-0.924291,-0.381689,-1.0,-1.836970e-16,-1.0,-1.836970e-16
8,0.0,0.833333,9/9/14,233.82,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,252,3,-0.930724,-0.365723,-1.0,-1.836970e-16,-1.0,-1.836970e-16
9,0.0,0.833333,9/10/14,229.74,29.02,31.64,29.57,30.73,29.71,31.52,...,2014,9,253,3,-0.936881,-0.349647,-1.0,-1.836970e-16,-1.0,-1.836970e-16


### REMOVE OUTLINER

In [21]:
# x = remove_outliers(x, relevant_features[3:-1], 4)

In [22]:
#Split x, y 
target = 'contest-tmp2m-14d__tmp2m'
drop_cols = ['DATE', 'startdate', target]
y = train[target]
X = train.drop([x for x in drop_cols if x in train.columns], axis=1)
X_test = test.drop([x for x in drop_cols if x in test.columns], axis=1)

In [23]:
#defining metric
def rmse(actual, predicted):
    return mean_squared_error(actual, predicted, squared=False)

In [24]:
X.columns.tolist()

['lat',
 'lon',
 'contest-pevpr-sfc-gauss-14d__pevpr',
 'nmme0-tmp2m-34w__cancm30',
 'nmme0-tmp2m-34w__cancm40',
 'nmme0-tmp2m-34w__ccsm30',
 'nmme0-tmp2m-34w__ccsm40',
 'nmme0-tmp2m-34w__cfsv20',
 'nmme0-tmp2m-34w__gfdlflora0',
 'nmme0-tmp2m-34w__gfdlflorb0',
 'nmme0-tmp2m-34w__gfdl0',
 'nmme0-tmp2m-34w__nasa0',
 'nmme0-tmp2m-34w__nmme0mean',
 'contest-wind-h10-14d__wind-hgt-10',
 'nmme-tmp2m-56w__cancm3',
 'nmme-tmp2m-56w__cancm4',
 'nmme-tmp2m-56w__ccsm3',
 'nmme-tmp2m-56w__ccsm4',
 'nmme-tmp2m-56w__cfsv2',
 'nmme-tmp2m-56w__gfdl',
 'nmme-tmp2m-56w__gfdlflora',
 'nmme-tmp2m-56w__gfdlflorb',
 'nmme-tmp2m-56w__nasa',
 'nmme-tmp2m-56w__nmmemean',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__cancm3',
 'nmme-prate-34w__cancm4',
 'nmme-prate-34w__ccsm3',
 'nmme-prate-34w__ccsm4',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__gfdl',
 'nmme-prate-34w__gfdlflora',
 'nmme-prate-34w__gfdlflorb',
 'nmme-prate-34w__nasa',
 'nmme-prate-34w__nmmemean',
 'contest-wind-h100-14d__wind-hgt-100',


In [25]:
# #Export to csv for reuse
# X.to_csv('train_cleaned.csv')
# X_test.to_csv('test_cleaned.csv')

# Feature Selection based on Random Forest Regressor

In [27]:
# from sklearn.ensemble import RandomForestRegressor 
# from sklearn.feature_selection import SelectFromModel

# from sklearn.feature_selection import SelectFromModel
# sel = SelectFromModel(RandomForestClassifier(n_estimators=100))
# sel.fit(X_train, y_train)



# # rf_pipeline = Pipeline([
# #   ('feature_selection', SelectFromModel(RandomForestRegressor(n_estimator=100))),
# #   ('classification', RandomForestClassifier())
# # ])

# rf_pipeline.fit(X_train, y_train)

In [ ]:
# sel.get_support()
# selected_feat= X.columns[(sel.get_support())]
# len(selected_feat)

# Modeling & Cross Validation 

In [28]:
#Train test
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)
print(f'Train_shape: {X_train.shape}    |   Val_shape: {X_val.shape}    |   Test_shape: {X_test.shape}')

Train_shape: (251741, 254)    |   Val_shape: (123993, 254)    |   Test_shape: (31354, 254)


# try feature selection by regression

In [29]:
def identify_correlated(df, threshold):
    corr_matrix = df.corr().abs()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    reduced_corr_matrix = corr_matrix.mask(mask)
    features_to_drop = [c for c in reduced_corr_matrix.columns if any(reduced_corr_matrix[c] > threshold)]
    return features_to_drop

features_to_drop = identify_correlated(train, .80)
print(len(features_to_drop))
print(features_to_drop)

105
['lat', 'contest-pevpr-sfc-gauss-14d__pevpr', 'nmme0-tmp2m-34w__cancm30', 'nmme0-tmp2m-34w__cancm40', 'nmme0-tmp2m-34w__ccsm30', 'nmme0-tmp2m-34w__ccsm40', 'nmme0-tmp2m-34w__cfsv20', 'nmme0-tmp2m-34w__gfdlflora0', 'nmme0-tmp2m-34w__gfdlflorb0', 'nmme0-tmp2m-34w__gfdl0', 'nmme0-tmp2m-34w__nasa0', 'nmme0-tmp2m-34w__nmme0mean', 'contest-wind-h10-14d__wind-hgt-10', 'nmme-tmp2m-56w__cancm3', 'nmme-tmp2m-56w__cancm4', 'nmme-tmp2m-56w__ccsm3', 'nmme-tmp2m-56w__ccsm4', 'nmme-tmp2m-56w__cfsv2', 'nmme-tmp2m-56w__gfdl', 'nmme-tmp2m-56w__gfdlflora', 'nmme-tmp2m-56w__gfdlflorb', 'nmme-tmp2m-56w__nasa', 'nmme-tmp2m-56w__nmmemean', 'nmme-prate-34w__cancm3', 'nmme-prate-34w__cancm4', 'nmme-prate-34w__ccsm3', 'nmme-prate-34w__ccsm4', 'nmme-prate-34w__cfsv2', 'nmme-prate-34w__gfdl', 'nmme-prate-34w__gfdlflora', 'nmme-prate-34w__gfdlflorb', 'nmme-prate-34w__nasa', 'nmme-prate-34w__nmmemean', 'contest-wind-h100-14d__wind-hgt-100', 'nmme0-prate-56w__cancm30', 'nmme0-prate-56w__cancm40', 'nmme0-prate-56

In [30]:
X_reduced = X.drop([col for col in features_to_drop if col in X.columns], axis=1)
X_test_reduced = X_test.drop([col for col in features_to_drop if col in X_test.columns], axis=1)

In [31]:
#Train test
from sklearn.model_selection import train_test_split
X_train_reduced, X_val_reduced, y_train_reduced, y_val_reduced = train_test_split(X_reduced, y, test_size=0.33, random_state=42)
print(f'Train_shape: {X_train_reduced.shape}    |   Val_shape: {X_val_reduced.shape}    |   Test_shape: {X_test_reduced.shape}')

Train_shape: (251741, 150)    |   Val_shape: (123993, 150)    |   Test_shape: (31354, 150)


In [ ]:
#pip install catboost

In [ ]:
#pip install xgboost

# Use Lightgbm

In [ ]:
#pip install lightgbm

In [82]:
import lightgbm as lgb

print("Beginning training and fitting lightgbm model")
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'max_depth': 8,
    'learning_rate': 0.03,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 50,
    'min_data_in_leaf': 100,
    'n_estimators': 15000,
}

# Create the LightGBM model object
reg_lgb = lgb.LGBMRegressor(**params)

# Fit the model to the training data
reg_lgb.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)])

Beginning training and fitting lightgbm model
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=50 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[1]	training's rmse: 9.5934	valid_1's rmse: 9.58656
[2]	training's rmse: 9.32533	valid_1's rmse: 9.31868
[3]	training's rmse: 9.06554	valid_1's rmse: 9.05932
[4]	training's rmse: 8.81372	valid_1's rmse: 8.80795
[5]	training's rmse: 8.56934	valid_1's rmse: 8.56381
[6]	training's rmse: 8.33297	valid_1's rmse: 8.32773
[7]	training's rmse: 8.10374	valid_1's rmse: 8.09896
[8]	training's rmse: 7.88212	valid_1's rmse: 7.87782
[9]	training's rmse: 7.66704	valid_1's rmse: 7.66313
[10]	

[144]	training's rmse: 1.22225	valid_1's rmse: 1.22975
[145]	training's rmse: 1.2174	valid_1's rmse: 1.22473
[146]	training's rmse: 1.2135	valid_1's rmse: 1.22088
[147]	training's rmse: 1.2081	valid_1's rmse: 1.21547
[148]	training's rmse: 1.20385	valid_1's rmse: 1.21123
[149]	training's rmse: 1.2004	valid_1's rmse: 1.2078
[150]	training's rmse: 1.19669	valid_1's rmse: 1.2041
[151]	training's rmse: 1.19346	valid_1's rmse: 1.20091
[152]	training's rmse: 1.18992	valid_1's rmse: 1.1973
[153]	training's rmse: 1.18596	valid_1's rmse: 1.19326
[154]	training's rmse: 1.18195	valid_1's rmse: 1.1893
[155]	training's rmse: 1.17861	valid_1's rmse: 1.18596
[156]	training's rmse: 1.17554	valid_1's rmse: 1.18288
[157]	training's rmse: 1.17125	valid_1's rmse: 1.17862
[158]	training's rmse: 1.1677	valid_1's rmse: 1.17506
[159]	training's rmse: 1.16326	valid_1's rmse: 1.17062
[160]	training's rmse: 1.15878	valid_1's rmse: 1.16618
[161]	training's rmse: 1.15566	valid_1's rmse: 1.16306
[162]	training's rm

[292]	training's rmse: 0.915995	valid_1's rmse: 0.925617
[293]	training's rmse: 0.914878	valid_1's rmse: 0.924484
[294]	training's rmse: 0.913872	valid_1's rmse: 0.923497
[295]	training's rmse: 0.912148	valid_1's rmse: 0.921813
[296]	training's rmse: 0.911184	valid_1's rmse: 0.920854
[297]	training's rmse: 0.910198	valid_1's rmse: 0.919879
[298]	training's rmse: 0.909353	valid_1's rmse: 0.919063
[299]	training's rmse: 0.908138	valid_1's rmse: 0.917868
[300]	training's rmse: 0.906892	valid_1's rmse: 0.916645
[301]	training's rmse: 0.905688	valid_1's rmse: 0.915432
[302]	training's rmse: 0.904508	valid_1's rmse: 0.914278
[303]	training's rmse: 0.90366	valid_1's rmse: 0.913502
[304]	training's rmse: 0.902453	valid_1's rmse: 0.912325
[305]	training's rmse: 0.90158	valid_1's rmse: 0.911478
[306]	training's rmse: 0.90029	valid_1's rmse: 0.910227
[307]	training's rmse: 0.899293	valid_1's rmse: 0.909229
[308]	training's rmse: 0.898024	valid_1's rmse: 0.907999
[309]	training's rmse: 0.896884	va

KeyboardInterrupt: 

In [83]:
# Create the LightGBM model object
reg_lgb = lgb.LGBMRegressor(**params)

# Fit the model to the training data
reg_lgb.fit(X, y, eval_set=[(X_train, y_train), (X_val, y_val)])
# Get feature importances
importance_scores = reg_lgb.feature_importances_
feature_importances = pd.DataFrame({'feature': X_train.columns, 'importance': importance_scores})

# Sort the features by importance score
feature_importances = feature_importances.sort_values('importance', ascending=False)

[1]	valid_0's rmse: 9.59353	valid_1's rmse: 9.58657
[2]	valid_0's rmse: 9.32559	valid_1's rmse: 9.31883
[3]	valid_0's rmse: 9.06597	valid_1's rmse: 9.05925
[4]	valid_0's rmse: 8.8146	valid_1's rmse: 8.80803
[5]	valid_0's rmse: 8.57047	valid_1's rmse: 8.56432
[6]	valid_0's rmse: 8.33408	valid_1's rmse: 8.32817
[7]	valid_0's rmse: 8.10522	valid_1's rmse: 8.09954
[8]	valid_0's rmse: 7.88371	valid_1's rmse: 7.87817
[9]	valid_0's rmse: 7.6687	valid_1's rmse: 7.66348
[10]	valid_0's rmse: 7.46035	valid_1's rmse: 7.45513
[11]	valid_0's rmse: 7.25847	valid_1's rmse: 7.25337
[12]	valid_0's rmse: 7.06277	valid_1's rmse: 7.05784
[13]	valid_0's rmse: 6.8734	valid_1's rmse: 6.8686
[14]	valid_0's rmse: 6.68968	valid_1's rmse: 6.685
[15]	valid_0's rmse: 6.51153	valid_1's rmse: 6.50703
[16]	valid_0's rmse: 6.33865	valid_1's rmse: 6.33427
[17]	valid_0's rmse: 6.17153	valid_1's rmse: 6.16753
[18]	valid_0's rmse: 6.00985	valid_1's rmse: 6.00603
[19]	valid_0's rmse: 5.85322	valid_1's rmse: 5.84936
[20]	val

[157]	valid_0's rmse: 1.17043	valid_1's rmse: 1.16643
[158]	valid_0's rmse: 1.16582	valid_1's rmse: 1.16181
[159]	valid_0's rmse: 1.16242	valid_1's rmse: 1.1584
[160]	valid_0's rmse: 1.15778	valid_1's rmse: 1.15373
[161]	valid_0's rmse: 1.15336	valid_1's rmse: 1.14926
[162]	valid_0's rmse: 1.15046	valid_1's rmse: 1.14635
[163]	valid_0's rmse: 1.14646	valid_1's rmse: 1.1423
[164]	valid_0's rmse: 1.14393	valid_1's rmse: 1.13976
[165]	valid_0's rmse: 1.14096	valid_1's rmse: 1.13679
[166]	valid_0's rmse: 1.13684	valid_1's rmse: 1.13263
[167]	valid_0's rmse: 1.13427	valid_1's rmse: 1.12996
[168]	valid_0's rmse: 1.13155	valid_1's rmse: 1.12725
[169]	valid_0's rmse: 1.12793	valid_1's rmse: 1.12364
[170]	valid_0's rmse: 1.12392	valid_1's rmse: 1.11959
[171]	valid_0's rmse: 1.12154	valid_1's rmse: 1.11719
[172]	valid_0's rmse: 1.11796	valid_1's rmse: 1.11358
[173]	valid_0's rmse: 1.11549	valid_1's rmse: 1.11112
[174]	valid_0's rmse: 1.11217	valid_1's rmse: 1.10775
[175]	valid_0's rmse: 1.10945	

[308]	valid_0's rmse: 0.895219	valid_1's rmse: 0.890645
[309]	valid_0's rmse: 0.89452	valid_1's rmse: 0.88991
[310]	valid_0's rmse: 0.893687	valid_1's rmse: 0.889075
[311]	valid_0's rmse: 0.892117	valid_1's rmse: 0.887493
[312]	valid_0's rmse: 0.89096	valid_1's rmse: 0.886338
[313]	valid_0's rmse: 0.889932	valid_1's rmse: 0.885339
[314]	valid_0's rmse: 0.888832	valid_1's rmse: 0.884217
[315]	valid_0's rmse: 0.888047	valid_1's rmse: 0.883449
[316]	valid_0's rmse: 0.887174	valid_1's rmse: 0.882596
[317]	valid_0's rmse: 0.886026	valid_1's rmse: 0.881443
[318]	valid_0's rmse: 0.885068	valid_1's rmse: 0.880517
[319]	valid_0's rmse: 0.884474	valid_1's rmse: 0.879914
[320]	valid_0's rmse: 0.883413	valid_1's rmse: 0.878864
[321]	valid_0's rmse: 0.882589	valid_1's rmse: 0.878063
[322]	valid_0's rmse: 0.881858	valid_1's rmse: 0.877332
[323]	valid_0's rmse: 0.880954	valid_1's rmse: 0.876444
[324]	valid_0's rmse: 0.879908	valid_1's rmse: 0.875398
[325]	valid_0's rmse: 0.879017	valid_1's rmse: 0.87

[457]	valid_0's rmse: 0.77497	valid_1's rmse: 0.770675
[458]	valid_0's rmse: 0.774396	valid_1's rmse: 0.770086
[459]	valid_0's rmse: 0.773623	valid_1's rmse: 0.769313
[460]	valid_0's rmse: 0.773013	valid_1's rmse: 0.768718
[461]	valid_0's rmse: 0.772394	valid_1's rmse: 0.768097
[462]	valid_0's rmse: 0.77194	valid_1's rmse: 0.767632
[463]	valid_0's rmse: 0.77136	valid_1's rmse: 0.767056
[464]	valid_0's rmse: 0.77047	valid_1's rmse: 0.766168
[465]	valid_0's rmse: 0.769435	valid_1's rmse: 0.765141
[466]	valid_0's rmse: 0.768729	valid_1's rmse: 0.764433
[467]	valid_0's rmse: 0.768027	valid_1's rmse: 0.763724
[468]	valid_0's rmse: 0.767434	valid_1's rmse: 0.763138
[469]	valid_0's rmse: 0.766831	valid_1's rmse: 0.762538
[470]	valid_0's rmse: 0.766261	valid_1's rmse: 0.76196
[471]	valid_0's rmse: 0.765653	valid_1's rmse: 0.76135
[472]	valid_0's rmse: 0.76516	valid_1's rmse: 0.76088
[473]	valid_0's rmse: 0.764586	valid_1's rmse: 0.760317
[474]	valid_0's rmse: 0.764021	valid_1's rmse: 0.759754


[605]	valid_0's rmse: 0.698823	valid_1's rmse: 0.694624
[606]	valid_0's rmse: 0.698555	valid_1's rmse: 0.694366
[607]	valid_0's rmse: 0.698168	valid_1's rmse: 0.693982
[608]	valid_0's rmse: 0.697845	valid_1's rmse: 0.693663
[609]	valid_0's rmse: 0.697208	valid_1's rmse: 0.693045
[610]	valid_0's rmse: 0.69679	valid_1's rmse: 0.692648
[611]	valid_0's rmse: 0.696387	valid_1's rmse: 0.692244
[612]	valid_0's rmse: 0.696	valid_1's rmse: 0.691882
[613]	valid_0's rmse: 0.695636	valid_1's rmse: 0.691532
[614]	valid_0's rmse: 0.695208	valid_1's rmse: 0.691098
[615]	valid_0's rmse: 0.694867	valid_1's rmse: 0.690745
[616]	valid_0's rmse: 0.694449	valid_1's rmse: 0.690343
[617]	valid_0's rmse: 0.693861	valid_1's rmse: 0.689765
[618]	valid_0's rmse: 0.693482	valid_1's rmse: 0.689392
[619]	valid_0's rmse: 0.693065	valid_1's rmse: 0.688987
[620]	valid_0's rmse: 0.692673	valid_1's rmse: 0.688603
[621]	valid_0's rmse: 0.692286	valid_1's rmse: 0.688214
[622]	valid_0's rmse: 0.69204	valid_1's rmse: 0.6879

...

[14327]	valid_0's rmse: 0.156558	valid_1's rmse: 0.15588
[14328]	valid_0's rmse: 0.156551	valid_1's rmse: 0.155873
[14329]	valid_0's rmse: 0.156542	valid_1's rmse: 0.155864
[14330]	valid_0's rmse: 0.156537	valid_1's rmse: 0.155858
[14331]	valid_0's rmse: 0.156528	valid_1's rmse: 0.155849
[14332]	valid_0's rmse: 0.15652	valid_1's rmse: 0.155842
[14333]	valid_0's rmse: 0.156514	valid_1's rmse: 0.155836
[14334]	valid_0's rmse: 0.156507	valid_1's rmse: 0.15583
[14335]	valid_0's rmse: 0.156501	valid_1's rmse: 0.155822
[14336]	valid_0's rmse: 0.156493	valid_1's rmse: 0.155813
[14337]	valid_0's rmse: 0.156485	valid_1's rmse: 0.155805
[14338]	valid_0's rmse: 0.156478	valid_1's rmse: 0.155798
[14339]	valid_0's rmse: 0.156472	valid_1's rmse: 0.155792
[14340]	valid_0's rmse: 0.156465	valid_1's rmse: 0.155785
[14341]	valid_0's rmse: 0.156459	valid_1's rmse: 0.15578
[14342]	valid_0's rmse: 0.156454	valid_1's rmse: 0.155775
[14343]	valid_0's rmse: 0.156448	valid_1's rmse: 0.15577
[14344]	valid_0's r

[14471]	valid_0's rmse: 0.155568	valid_1's rmse: 0.154889
[14472]	valid_0's rmse: 0.155561	valid_1's rmse: 0.154882
[14473]	valid_0's rmse: 0.155557	valid_1's rmse: 0.154877
[14474]	valid_0's rmse: 0.155547	valid_1's rmse: 0.154869
[14475]	valid_0's rmse: 0.15554	valid_1's rmse: 0.154862
[14476]	valid_0's rmse: 0.155534	valid_1's rmse: 0.154856
[14477]	valid_0's rmse: 0.155528	valid_1's rmse: 0.15485
[14478]	valid_0's rmse: 0.155522	valid_1's rmse: 0.154844
[14479]	valid_0's rmse: 0.155516	valid_1's rmse: 0.154837
[14480]	valid_0's rmse: 0.155511	valid_1's rmse: 0.154831
[14481]	valid_0's rmse: 0.155506	valid_1's rmse: 0.154825
[14482]	valid_0's rmse: 0.155501	valid_1's rmse: 0.154819
[14483]	valid_0's rmse: 0.155494	valid_1's rmse: 0.154811
[14484]	valid_0's rmse: 0.155486	valid_1's rmse: 0.154804
[14485]	valid_0's rmse: 0.155481	valid_1's rmse: 0.154799
[14486]	valid_0's rmse: 0.155471	valid_1's rmse: 0.15479
[14487]	valid_0's rmse: 0.155464	valid_1's rmse: 0.154782
[14488]	valid_0's

[14616]	valid_0's rmse: 0.154634	valid_1's rmse: 0.153968
[14617]	valid_0's rmse: 0.154627	valid_1's rmse: 0.153961
[14618]	valid_0's rmse: 0.154622	valid_1's rmse: 0.153956
[14619]	valid_0's rmse: 0.154614	valid_1's rmse: 0.153948
[14620]	valid_0's rmse: 0.154607	valid_1's rmse: 0.153942
[14621]	valid_0's rmse: 0.154602	valid_1's rmse: 0.153937
[14622]	valid_0's rmse: 0.154596	valid_1's rmse: 0.153931
[14623]	valid_0's rmse: 0.154591	valid_1's rmse: 0.153926
[14624]	valid_0's rmse: 0.154586	valid_1's rmse: 0.15392
[14625]	valid_0's rmse: 0.154581	valid_1's rmse: 0.153916
[14626]	valid_0's rmse: 0.154573	valid_1's rmse: 0.153909
[14627]	valid_0's rmse: 0.154567	valid_1's rmse: 0.153902
[14628]	valid_0's rmse: 0.154561	valid_1's rmse: 0.153895
[14629]	valid_0's rmse: 0.154555	valid_1's rmse: 0.15389
[14630]	valid_0's rmse: 0.154547	valid_1's rmse: 0.153882
[14631]	valid_0's rmse: 0.154538	valid_1's rmse: 0.153874
[14632]	valid_0's rmse: 0.15453	valid_1's rmse: 0.153865
[14633]	valid_0's

[14758]	valid_0's rmse: 0.153675	valid_1's rmse: 0.153016
[14759]	valid_0's rmse: 0.15367	valid_1's rmse: 0.153011
[14760]	valid_0's rmse: 0.153665	valid_1's rmse: 0.153006
[14761]	valid_0's rmse: 0.153659	valid_1's rmse: 0.152999
[14762]	valid_0's rmse: 0.153652	valid_1's rmse: 0.152993
[14763]	valid_0's rmse: 0.153649	valid_1's rmse: 0.152989
[14764]	valid_0's rmse: 0.153642	valid_1's rmse: 0.152983
[14765]	valid_0's rmse: 0.153636	valid_1's rmse: 0.152978
[14766]	valid_0's rmse: 0.153631	valid_1's rmse: 0.152973
[14767]	valid_0's rmse: 0.153627	valid_1's rmse: 0.152969
[14768]	valid_0's rmse: 0.153622	valid_1's rmse: 0.152964
[14769]	valid_0's rmse: 0.153617	valid_1's rmse: 0.152959
[14770]	valid_0's rmse: 0.153613	valid_1's rmse: 0.152955
[14771]	valid_0's rmse: 0.153607	valid_1's rmse: 0.152947
[14772]	valid_0's rmse: 0.1536	valid_1's rmse: 0.152941
[14773]	valid_0's rmse: 0.153593	valid_1's rmse: 0.152934
[14774]	valid_0's rmse: 0.153588	valid_1's rmse: 0.152928
[14775]	valid_0's

[14902]	valid_0's rmse: 0.15281	valid_1's rmse: 0.152159
[14903]	valid_0's rmse: 0.152804	valid_1's rmse: 0.152153
[14904]	valid_0's rmse: 0.152796	valid_1's rmse: 0.152147
[14905]	valid_0's rmse: 0.15279	valid_1's rmse: 0.152141
[14906]	valid_0's rmse: 0.152785	valid_1's rmse: 0.152136
[14907]	valid_0's rmse: 0.15278	valid_1's rmse: 0.152131
[14908]	valid_0's rmse: 0.152775	valid_1's rmse: 0.152127
[14909]	valid_0's rmse: 0.15277	valid_1's rmse: 0.152122
[14910]	valid_0's rmse: 0.152766	valid_1's rmse: 0.152119
[14911]	valid_0's rmse: 0.152762	valid_1's rmse: 0.152115
[14912]	valid_0's rmse: 0.152757	valid_1's rmse: 0.15211
[14913]	valid_0's rmse: 0.152751	valid_1's rmse: 0.152105
[14914]	valid_0's rmse: 0.152747	valid_1's rmse: 0.152101
[14915]	valid_0's rmse: 0.152742	valid_1's rmse: 0.152096
[14916]	valid_0's rmse: 0.152737	valid_1's rmse: 0.15209
[14917]	valid_0's rmse: 0.152732	valid_1's rmse: 0.152085
[14918]	valid_0's rmse: 0.152728	valid_1's rmse: 0.15208
[14919]	valid_0's rms

In [84]:
feature_importances

,feature,importance
243,loc_group,12607
68,contest-pres-sfc-gauss-14d__pres,10896
1,lon,8072
85,contest-wind-h500-14d__wind-hgt-500,6740
97,elevation__elevation,6657
...,...,...
244,year,24
245,month,19
247,season,7
253,season_cos,1


In [34]:
#LightGBM for reduced dataset 
import lightgbm as lgb

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'max_depth': 8,
    'learning_rate': 0.03,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 50,
    'min_data_in_leaf': 100,
    'n_estimators': 15000,
}

# Create the LightGBM model object
reg_lgb_r = lgb.LGBMRegressor(**params)

# Fit the model to the training data
reg_lgb_r.fit(X_reduced, y, eval_set=[(X_train_reduced, y_train_reduced), (X_val_reduced, y_val_reduced)])

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=50 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[1]	valid_0's rmse: 9.60407	valid_1's rmse: 9.59748
[2]	valid_0's rmse: 9.34641	valid_1's rmse: 9.34037
[3]	valid_0's rmse: 9.09849	valid_1's rmse: 9.09277
[4]	valid_0's rmse: 8.85678	valid_1's rmse: 8.85156
[5]	valid_0's rmse: 8.62281	valid_1's rmse: 8.61792
[6]	valid_0's rmse: 8.39497	valid_1's rmse: 8.39078
[7]	valid_0's rmse: 8.1756	valid_1's rmse: 8.17171
[8]	valid_0's rmse: 7.96147	valid_1's rmse: 7.95799
[9]	valid_0's rmse: 7.7547	valid_1's rmse: 7.75161
[10]	valid_0's rmse: 7.55369	valid_1's rmse: 7.55091
[11]	val

[150]	valid_0's rmse: 1.38492	valid_1's rmse: 1.38388
[151]	valid_0's rmse: 1.38016	valid_1's rmse: 1.37911
[152]	valid_0's rmse: 1.37549	valid_1's rmse: 1.37441
[153]	valid_0's rmse: 1.3698	valid_1's rmse: 1.36866
[154]	valid_0's rmse: 1.36517	valid_1's rmse: 1.36402
[155]	valid_0's rmse: 1.36096	valid_1's rmse: 1.35978
[156]	valid_0's rmse: 1.35732	valid_1's rmse: 1.35603
[157]	valid_0's rmse: 1.35359	valid_1's rmse: 1.35225
[158]	valid_0's rmse: 1.35052	valid_1's rmse: 1.34913
[159]	valid_0's rmse: 1.34646	valid_1's rmse: 1.34497
[160]	valid_0's rmse: 1.34225	valid_1's rmse: 1.34065
[161]	valid_0's rmse: 1.33811	valid_1's rmse: 1.33655
[162]	valid_0's rmse: 1.33553	valid_1's rmse: 1.33397
[163]	valid_0's rmse: 1.33243	valid_1's rmse: 1.33076
[164]	valid_0's rmse: 1.32917	valid_1's rmse: 1.32749
[165]	valid_0's rmse: 1.32613	valid_1's rmse: 1.32444
[166]	valid_0's rmse: 1.32281	valid_1's rmse: 1.32109
[167]	valid_0's rmse: 1.31905	valid_1's rmse: 1.3173
[168]	valid_0's rmse: 1.31653	

[308]	valid_0's rmse: 1.07038	valid_1's rmse: 1.06696
[309]	valid_0's rmse: 1.06854	valid_1's rmse: 1.06509
[310]	valid_0's rmse: 1.06762	valid_1's rmse: 1.06416
[311]	valid_0's rmse: 1.06657	valid_1's rmse: 1.06303
[312]	valid_0's rmse: 1.06566	valid_1's rmse: 1.06211
[313]	valid_0's rmse: 1.06437	valid_1's rmse: 1.06079
[314]	valid_0's rmse: 1.06354	valid_1's rmse: 1.05998
[315]	valid_0's rmse: 1.06278	valid_1's rmse: 1.05923
[316]	valid_0's rmse: 1.06105	valid_1's rmse: 1.05752
[317]	valid_0's rmse: 1.05996	valid_1's rmse: 1.05649
[318]	valid_0's rmse: 1.05922	valid_1's rmse: 1.05572
[319]	valid_0's rmse: 1.05815	valid_1's rmse: 1.05466
[320]	valid_0's rmse: 1.05572	valid_1's rmse: 1.05223
[321]	valid_0's rmse: 1.05402	valid_1's rmse: 1.05048
[322]	valid_0's rmse: 1.05334	valid_1's rmse: 1.04978
[323]	valid_0's rmse: 1.05259	valid_1's rmse: 1.04906
[324]	valid_0's rmse: 1.05195	valid_1's rmse: 1.04842
[325]	valid_0's rmse: 1.05143	valid_1's rmse: 1.04789
[326]	valid_0's rmse: 1.0498

[461]	valid_0's rmse: 0.925061	valid_1's rmse: 0.921388
[462]	valid_0's rmse: 0.924448	valid_1's rmse: 0.92079
[463]	valid_0's rmse: 0.923865	valid_1's rmse: 0.92022
[464]	valid_0's rmse: 0.923318	valid_1's rmse: 0.919673
[465]	valid_0's rmse: 0.922176	valid_1's rmse: 0.918533
[466]	valid_0's rmse: 0.921431	valid_1's rmse: 0.917765
[467]	valid_0's rmse: 0.920648	valid_1's rmse: 0.916975
[468]	valid_0's rmse: 0.919972	valid_1's rmse: 0.916262
[469]	valid_0's rmse: 0.919461	valid_1's rmse: 0.915735
[470]	valid_0's rmse: 0.919067	valid_1's rmse: 0.915368
[471]	valid_0's rmse: 0.918525	valid_1's rmse: 0.914821
[472]	valid_0's rmse: 0.918034	valid_1's rmse: 0.914321
[473]	valid_0's rmse: 0.917599	valid_1's rmse: 0.913883
[474]	valid_0's rmse: 0.917204	valid_1's rmse: 0.91349
[475]	valid_0's rmse: 0.915713	valid_1's rmse: 0.912013
[476]	valid_0's rmse: 0.915085	valid_1's rmse: 0.911394
[477]	valid_0's rmse: 0.914646	valid_1's rmse: 0.910952
[478]	valid_0's rmse: 0.913952	valid_1's rmse: 0.91

[609]	valid_0's rmse: 0.839556	valid_1's rmse: 0.835845
[610]	valid_0's rmse: 0.839215	valid_1's rmse: 0.835506
[611]	valid_0's rmse: 0.838901	valid_1's rmse: 0.835189
[612]	valid_0's rmse: 0.838624	valid_1's rmse: 0.83492
[613]	valid_0's rmse: 0.837208	valid_1's rmse: 0.833497
[614]	valid_0's rmse: 0.836762	valid_1's rmse: 0.833061
[615]	valid_0's rmse: 0.836221	valid_1's rmse: 0.832535
[616]	valid_0's rmse: 0.835877	valid_1's rmse: 0.832192
[617]	valid_0's rmse: 0.835404	valid_1's rmse: 0.831707
[618]	valid_0's rmse: 0.834864	valid_1's rmse: 0.831166
[619]	valid_0's rmse: 0.834461	valid_1's rmse: 0.830764
[620]	valid_0's rmse: 0.834149	valid_1's rmse: 0.830448
[621]	valid_0's rmse: 0.833848	valid_1's rmse: 0.830155
[622]	valid_0's rmse: 0.832611	valid_1's rmse: 0.828941
[623]	valid_0's rmse: 0.832048	valid_1's rmse: 0.828375
[624]	valid_0's rmse: 0.830912	valid_1's rmse: 0.827253
[625]	valid_0's rmse: 0.830572	valid_1's rmse: 0.826922
[626]	valid_0's rmse: 0.830053	valid_1's rmse: 0.

...

[14557]	valid_0's rmse: 0.199269	valid_1's rmse: 0.199493
[14558]	valid_0's rmse: 0.199248	valid_1's rmse: 0.199469
[14559]	valid_0's rmse: 0.199242	valid_1's rmse: 0.199463
[14560]	valid_0's rmse: 0.199236	valid_1's rmse: 0.199455
[14561]	valid_0's rmse: 0.199228	valid_1's rmse: 0.199449
[14562]	valid_0's rmse: 0.199222	valid_1's rmse: 0.199444
[14563]	valid_0's rmse: 0.199214	valid_1's rmse: 0.199438
[14564]	valid_0's rmse: 0.199209	valid_1's rmse: 0.199433
[14565]	valid_0's rmse: 0.199204	valid_1's rmse: 0.199428
[14566]	valid_0's rmse: 0.199195	valid_1's rmse: 0.199419
[14567]	valid_0's rmse: 0.199188	valid_1's rmse: 0.199411
[14568]	valid_0's rmse: 0.19918	valid_1's rmse: 0.199403
[14569]	valid_0's rmse: 0.199171	valid_1's rmse: 0.199395
[14570]	valid_0's rmse: 0.199165	valid_1's rmse: 0.199389
[14571]	valid_0's rmse: 0.199156	valid_1's rmse: 0.199383
[14572]	valid_0's rmse: 0.199151	valid_1's rmse: 0.199377
[14573]	valid_0's rmse: 0.199145	valid_1's rmse: 0.199372
[14574]	valid_0

[14705]	valid_0's rmse: 0.198141	valid_1's rmse: 0.19835
[14706]	valid_0's rmse: 0.198124	valid_1's rmse: 0.198332
[14707]	valid_0's rmse: 0.198113	valid_1's rmse: 0.198321
[14708]	valid_0's rmse: 0.198101	valid_1's rmse: 0.198311
[14709]	valid_0's rmse: 0.198082	valid_1's rmse: 0.198291
[14710]	valid_0's rmse: 0.198074	valid_1's rmse: 0.198284
[14711]	valid_0's rmse: 0.198067	valid_1's rmse: 0.198277
[14712]	valid_0's rmse: 0.19806	valid_1's rmse: 0.19827
[14713]	valid_0's rmse: 0.198052	valid_1's rmse: 0.198264
[14714]	valid_0's rmse: 0.198047	valid_1's rmse: 0.198258
[14715]	valid_0's rmse: 0.198042	valid_1's rmse: 0.198253
[14716]	valid_0's rmse: 0.198036	valid_1's rmse: 0.198248
[14717]	valid_0's rmse: 0.198031	valid_1's rmse: 0.198243
[14718]	valid_0's rmse: 0.198025	valid_1's rmse: 0.198238
[14719]	valid_0's rmse: 0.198019	valid_1's rmse: 0.198233
[14720]	valid_0's rmse: 0.198007	valid_1's rmse: 0.198224
[14721]	valid_0's rmse: 0.197994	valid_1's rmse: 0.198211
[14722]	valid_0's

[14852]	valid_0's rmse: 0.196987	valid_1's rmse: 0.197182
[14853]	valid_0's rmse: 0.196979	valid_1's rmse: 0.197173
[14854]	valid_0's rmse: 0.196972	valid_1's rmse: 0.197163
[14855]	valid_0's rmse: 0.196965	valid_1's rmse: 0.197157
[14856]	valid_0's rmse: 0.196958	valid_1's rmse: 0.197151
[14857]	valid_0's rmse: 0.196953	valid_1's rmse: 0.197146
[14858]	valid_0's rmse: 0.196948	valid_1's rmse: 0.197139
[14859]	valid_0's rmse: 0.196942	valid_1's rmse: 0.197133
[14860]	valid_0's rmse: 0.196934	valid_1's rmse: 0.197127
[14861]	valid_0's rmse: 0.196929	valid_1's rmse: 0.197123
[14862]	valid_0's rmse: 0.196924	valid_1's rmse: 0.197118
[14863]	valid_0's rmse: 0.196919	valid_1's rmse: 0.197113
[14864]	valid_0's rmse: 0.196911	valid_1's rmse: 0.197107
[14865]	valid_0's rmse: 0.196907	valid_1's rmse: 0.197102
[14866]	valid_0's rmse: 0.196899	valid_1's rmse: 0.197095
[14867]	valid_0's rmse: 0.196894	valid_1's rmse: 0.197089
[14868]	valid_0's rmse: 0.196888	valid_1's rmse: 0.197083
[14869]	valid_

[15000]	valid_0's rmse: 0.195858	valid_1's rmse: 0.196033


LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, feature_fraction=0.9,
              learning_rate=0.03, max_depth=8, metric='rmse',
              min_child_samples=50, min_data_in_leaf=100, n_estimators=15000,
              objective='regression')

In [ ]:
# Generate predictions on pseudo-eval set (splitted from train set)
# Create the LightGBM model object
# Fit the model to the training data
#reg_lgb_1.fit((X, y))

In [88]:
pip install optuna

     -------------------------------------- 365.3/365.3 kB 2.8 MB/s eta 0:00:00
     ------------------------------------- 210.5/210.5 kB 12.5 MB/s eta 0:00:00
     ---------------------------------------- 78.7/78.7 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import optuna 

def objective(trial):
    params = {
        'boosting_type': 'gbdt', 
        'objective': 'regression', 
        'metric': 'rmse', 
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'lambda_l1': trial.suggest_int('lambda_l1', 0, 100),
        'lambda_l2': trial.suggest_int('lambda_l2', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'device_type':'gpu',
        'verbosity': -1,
        'n_estimators': 6000
    }

    reg_lgb_2 = lgb.LGBMRegressor(**params)

    reg_lgb_2.fit(X_train_reduced, y_train_reduced, eval_set=[(X_train_reduced, y_train_reduced), (X_val_reduced, y_val_reduced)])

    y_pred_lgb_2 = reg_lgb_2.predict(X_val_reduced)

    rmse = np.sqrt(mean_squared_error(y_val_reduced, y_pred_lgb_op))

    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=15)

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print(f'  Value: {trial.value:.5f}')
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')    

best_params = trial.params
reg_lgb_op = lgb.LGBMRegressor(**best_params)
reg_lgb_op.fit(X_train_reduced, y_train_reduced, eval_set=[(X_train_reduced, y_train_reduced), (X_val_reduced, y_val_reduced)])
y_pred_lgb_op = reg_lgb.predict(X_val_reduced)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_lgb))
print("RMSE:", rmse)

[I 2023-02-28 12:21:28,231] A new study created in memory with name: no-name-3d6c9cdc-377d-498c-88e4-d72003477ad7
C:\Users\Lab N03\AppData\Local\Temp\ipykernel_9728\1291196895.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),


[1]	training's rmse: 9.77051	valid_1's rmse: 9.76376
[2]	training's rmse: 9.67204	valid_1's rmse: 9.66552
[3]	training's rmse: 9.57474	valid_1's rmse: 9.56845
[4]	training's rmse: 9.47864	valid_1's rmse: 9.47257
[5]	training's rmse: 9.38363	valid_1's rmse: 9.37778
[6]	training's rmse: 9.2897	valid_1's rmse: 9.2841
[7]	training's rmse: 9.19694	valid_1's rmse: 9.1915
[8]	training's rmse: 9.10521	valid_1's rmse: 9.09999
[9]	training's rmse: 9.01469	valid_1's rmse: 9.00967
[10]	training's rmse: 8.92482	valid_1's rmse: 8.92005
[11]	training's rmse: 8.8361	valid_1's rmse: 8.83151
[12]	training's rmse: 8.74839	valid_1's rmse: 8.74403
[13]	training's rmse: 8.66176	valid_1's rmse: 8.65748
[14]	training's rmse: 8.57616	valid_1's rmse: 8.57207
[15]	training's rmse: 8.49132	valid_1's rmse: 8.48744
[16]	training's rmse: 8.40749	valid_1's rmse: 8.4038
[17]	training's rmse: 8.32462	valid_1's rmse: 8.32115
[18]	training's rmse: 8.24276	valid_1's rmse: 8.2394
[19]	training's rmse: 8.16173	valid_1's rms

[154]	training's rmse: 2.67833	valid_1's rmse: 2.69241
[155]	training's rmse: 2.66223	valid_1's rmse: 2.67634
[156]	training's rmse: 2.64623	valid_1's rmse: 2.66047
[157]	training's rmse: 2.63064	valid_1's rmse: 2.64491
[158]	training's rmse: 2.6147	valid_1's rmse: 2.62905
[159]	training's rmse: 2.59909	valid_1's rmse: 2.61351
[160]	training's rmse: 2.58417	valid_1's rmse: 2.5986
[161]	training's rmse: 2.56948	valid_1's rmse: 2.58396
[162]	training's rmse: 2.55466	valid_1's rmse: 2.56916
[163]	training's rmse: 2.53999	valid_1's rmse: 2.5546
[164]	training's rmse: 2.5254	valid_1's rmse: 2.54005
[165]	training's rmse: 2.51113	valid_1's rmse: 2.52589
[166]	training's rmse: 2.49674	valid_1's rmse: 2.51153
[167]	training's rmse: 2.48297	valid_1's rmse: 2.4978
[168]	training's rmse: 2.469	valid_1's rmse: 2.48386
[169]	training's rmse: 2.45563	valid_1's rmse: 2.47049
[170]	training's rmse: 2.44197	valid_1's rmse: 2.45688
[171]	training's rmse: 2.42879	valid_1's rmse: 2.44371
[172]	training's 

[304]	training's rmse: 1.54808	valid_1's rmse: 1.56308
[305]	training's rmse: 1.54498	valid_1's rmse: 1.56001
[306]	training's rmse: 1.54216	valid_1's rmse: 1.55715
[307]	training's rmse: 1.53878	valid_1's rmse: 1.55376
[308]	training's rmse: 1.53594	valid_1's rmse: 1.55091
[309]	training's rmse: 1.53266	valid_1's rmse: 1.54761
[310]	training's rmse: 1.5299	valid_1's rmse: 1.5448
[311]	training's rmse: 1.52667	valid_1's rmse: 1.54155
[312]	training's rmse: 1.52414	valid_1's rmse: 1.53903
[313]	training's rmse: 1.52111	valid_1's rmse: 1.53599
[314]	training's rmse: 1.5186	valid_1's rmse: 1.53348
[315]	training's rmse: 1.51564	valid_1's rmse: 1.53052
[316]	training's rmse: 1.51319	valid_1's rmse: 1.52807
[317]	training's rmse: 1.51056	valid_1's rmse: 1.52542
[318]	training's rmse: 1.50763	valid_1's rmse: 1.52246
[319]	training's rmse: 1.50482	valid_1's rmse: 1.51964
[320]	training's rmse: 1.50253	valid_1's rmse: 1.51734
[321]	training's rmse: 1.49984	valid_1's rmse: 1.51462
[322]	trainin

[455]	training's rmse: 1.26926	valid_1's rmse: 1.28263
[456]	training's rmse: 1.26789	valid_1's rmse: 1.28127
[457]	training's rmse: 1.2671	valid_1's rmse: 1.28049
[458]	training's rmse: 1.26608	valid_1's rmse: 1.27946
[459]	training's rmse: 1.26533	valid_1's rmse: 1.27873
[460]	training's rmse: 1.26394	valid_1's rmse: 1.27732
[461]	training's rmse: 1.26287	valid_1's rmse: 1.27626
[462]	training's rmse: 1.26182	valid_1's rmse: 1.27522
[463]	training's rmse: 1.26091	valid_1's rmse: 1.27431
[464]	training's rmse: 1.26009	valid_1's rmse: 1.27348
[465]	training's rmse: 1.25878	valid_1's rmse: 1.27218
[466]	training's rmse: 1.25761	valid_1's rmse: 1.27099
[467]	training's rmse: 1.25647	valid_1's rmse: 1.26985
[468]	training's rmse: 1.25564	valid_1's rmse: 1.26903
[469]	training's rmse: 1.25452	valid_1's rmse: 1.26791
[470]	training's rmse: 1.25373	valid_1's rmse: 1.26711
[471]	training's rmse: 1.25255	valid_1's rmse: 1.2659
[472]	training's rmse: 1.2516	valid_1's rmse: 1.26496
[473]	trainin

...

[1770]	training's rmse: 0.76256	valid_1's rmse: 0.780242
[1771]	training's rmse: 0.762448	valid_1's rmse: 0.780134
[1772]	training's rmse: 0.762347	valid_1's rmse: 0.780041
[1773]	training's rmse: 0.762231	valid_1's rmse: 0.779924
[1774]	training's rmse: 0.762052	valid_1's rmse: 0.779745
[1775]	training's rmse: 0.761918	valid_1's rmse: 0.77962
[1776]	training's rmse: 0.76177	valid_1's rmse: 0.779478
[1777]	training's rmse: 0.761655	valid_1's rmse: 0.779368
[1778]	training's rmse: 0.761538	valid_1's rmse: 0.779249
[1779]	training's rmse: 0.761079	valid_1's rmse: 0.778786
[1780]	training's rmse: 0.760913	valid_1's rmse: 0.778628
[1781]	training's rmse: 0.760824	valid_1's rmse: 0.778543
[1782]	training's rmse: 0.760652	valid_1's rmse: 0.778377
[1783]	training's rmse: 0.760544	valid_1's rmse: 0.778275
[1784]	training's rmse: 0.760442	valid_1's rmse: 0.77817
[1785]	training's rmse: 0.760296	valid_1's rmse: 0.778032
[1786]	training's rmse: 0.760187	valid_1's rmse: 0.777926
[1787]	training's 

[1914]	training's rmse: 0.740333	valid_1's rmse: 0.758597
[1915]	training's rmse: 0.740168	valid_1's rmse: 0.758436
[1916]	training's rmse: 0.740044	valid_1's rmse: 0.758321
[1917]	training's rmse: 0.73988	valid_1's rmse: 0.758161
[1918]	training's rmse: 0.739791	valid_1's rmse: 0.758072
[1919]	training's rmse: 0.739657	valid_1's rmse: 0.757945
[1920]	training's rmse: 0.739431	valid_1's rmse: 0.757737
[1921]	training's rmse: 0.7393	valid_1's rmse: 0.757611
[1922]	training's rmse: 0.739204	valid_1's rmse: 0.757518
[1923]	training's rmse: 0.739106	valid_1's rmse: 0.757425
[1924]	training's rmse: 0.73893	valid_1's rmse: 0.757255
[1925]	training's rmse: 0.738827	valid_1's rmse: 0.757151
[1926]	training's rmse: 0.73874	valid_1's rmse: 0.757069
[1927]	training's rmse: 0.738554	valid_1's rmse: 0.756897
[1928]	training's rmse: 0.738335	valid_1's rmse: 0.756696
[1929]	training's rmse: 0.73822	valid_1's rmse: 0.756582
[1930]	training's rmse: 0.738078	valid_1's rmse: 0.756444
[1931]	training's rm

[2059]	training's rmse: 0.719894	valid_1's rmse: 0.738835
[2060]	training's rmse: 0.719816	valid_1's rmse: 0.738761
[2061]	training's rmse: 0.719722	valid_1's rmse: 0.738671
[2062]	training's rmse: 0.719641	valid_1's rmse: 0.738598
[2063]	training's rmse: 0.719404	valid_1's rmse: 0.738372
[2064]	training's rmse: 0.7193	valid_1's rmse: 0.738275
[2065]	training's rmse: 0.719213	valid_1's rmse: 0.738193
[2066]	training's rmse: 0.71911	valid_1's rmse: 0.738096
[2067]	training's rmse: 0.719013	valid_1's rmse: 0.738006
[2068]	training's rmse: 0.718934	valid_1's rmse: 0.73793
[2069]	training's rmse: 0.718839	valid_1's rmse: 0.737842
[2070]	training's rmse: 0.718696	valid_1's rmse: 0.737697
[2071]	training's rmse: 0.718596	valid_1's rmse: 0.737603
[2072]	training's rmse: 0.718431	valid_1's rmse: 0.737436
[2073]	training's rmse: 0.718341	valid_1's rmse: 0.737355
[2074]	training's rmse: 0.718233	valid_1's rmse: 0.737251
[2075]	training's rmse: 0.718075	valid_1's rmse: 0.737113
[2076]	training's 

[2204]	training's rmse: 0.703427	valid_1's rmse: 0.723046
[2205]	training's rmse: 0.703232	valid_1's rmse: 0.72286
[2206]	training's rmse: 0.703156	valid_1's rmse: 0.722789
[2207]	training's rmse: 0.703071	valid_1's rmse: 0.722713
[2208]	training's rmse: 0.702998	valid_1's rmse: 0.722649
[2209]	training's rmse: 0.702916	valid_1's rmse: 0.72257
[2210]	training's rmse: 0.702722	valid_1's rmse: 0.722376
[2211]	training's rmse: 0.7026	valid_1's rmse: 0.722263
[2212]	training's rmse: 0.702519	valid_1's rmse: 0.722185
[2213]	training's rmse: 0.702396	valid_1's rmse: 0.722068
[2214]	training's rmse: 0.702266	valid_1's rmse: 0.72194
[2215]	training's rmse: 0.702131	valid_1's rmse: 0.721804
[2216]	training's rmse: 0.701991	valid_1's rmse: 0.721664
[2217]	training's rmse: 0.70193	valid_1's rmse: 0.721605
[2218]	training's rmse: 0.701833	valid_1's rmse: 0.721512
[2219]	training's rmse: 0.701692	valid_1's rmse: 0.721377
[2220]	training's rmse: 0.701616	valid_1's rmse: 0.721304
[2221]	training's rm

[2346]	training's rmse: 0.688153	valid_1's rmse: 0.708345
[2347]	training's rmse: 0.688071	valid_1's rmse: 0.70827
[2348]	training's rmse: 0.687951	valid_1's rmse: 0.708157
[2349]	training's rmse: 0.687782	valid_1's rmse: 0.707988
[2350]	training's rmse: 0.687658	valid_1's rmse: 0.707869
[2351]	training's rmse: 0.687532	valid_1's rmse: 0.707751
[2352]	training's rmse: 0.687462	valid_1's rmse: 0.707682
[2353]	training's rmse: 0.68734	valid_1's rmse: 0.707564
[2354]	training's rmse: 0.687223	valid_1's rmse: 0.70745
[2355]	training's rmse: 0.687128	valid_1's rmse: 0.707355
[2356]	training's rmse: 0.687013	valid_1's rmse: 0.707251
[2357]	training's rmse: 0.686942	valid_1's rmse: 0.707186
[2358]	training's rmse: 0.686803	valid_1's rmse: 0.707053
[2359]	training's rmse: 0.686676	valid_1's rmse: 0.706934
[2360]	training's rmse: 0.686555	valid_1's rmse: 0.706818
[2361]	training's rmse: 0.686429	valid_1's rmse: 0.706693
[2362]	training's rmse: 0.686308	valid_1's rmse: 0.706581
[2363]	training's

In [86]:
#train model on full data
y_pred_lgb_2 = reg_lgb.predict(X_test)

In [87]:
submission = pd.read_csv('sample_solution.csv')
submission[target] = y_pred_lgb_2
submission.to_csv('submission_lgb_2.csv', 
                  index = False)

## Use MLP

In [ ]:
!pip install sklearn

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:

# Standardize the data using a standard scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

mlp_clf = MLPRegressor(hidden_layer_sizes=(100,200),
                        max_iter = 300,activation = 'relu',
                        solver = 'adam')

mlp_clf.fit(X_train, y_train)

# make predictions on the test data


In [ ]:
y_pred_mlp = mlp_clf.predict(X_train)

In [ ]:
rmse(y_train,y_pred_mlp)

In [ ]:
#Run feature selection by RFR
from sklearn.feature_selection import SelectFromModel
sel = SelectFromModel(RandomForestRegressor(n_estimators=200))
sel.fit(X, y)
sel.get_support()
selected_feat= X.columns[(sel.get_support())]
print(len(selected_feat), '/n', selected_feat)

# Use Neural network

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [ ]:
nn = Pipeline([('scaler',StandardScaler()),('net', MLPClassifier((10, 1), learning_rate_init = 0.0001, max_iter = 20000))])


In [ ]:
nn.fit(X, y)

In [ ]:
# Applying confusion matrix on the training set
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
yfit = nn.predict(X_train)
print('Accuracy: ' + str(accuracy_score(y_pred=y_val, y_true=y_train)))
print('Percision: ' + str(precision_score(y_pred=y_val, y_true=y_train)))

confusion_matrix = confusion_matrix(y_train, y_var, labels=nn.classes_)
print(confusion_matrix)
cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

cm_display.plot()
plt.show()

# Ensembling prediction

In [ ]:
import xgboost as xgb

reg_xgb = xgb.XGBRegressor(base_score=0.5, 
                           n_estimators=1500, 
                           objective='reg:squarederror',
                           max_depth=5,
                           early_stopping_rounds=20,
                           learning_rate=0.03)

reg_xgb.fit(X_train, y_train, eval_set=((X_val, y_val),), verbose=100)

# Hyperparameter tuning

# Train on full data

#Submission

#Reference